2.	Подготовьте sentiment analysis по отзывам (используя  nltk.classify) и оцените, как распределяется sentiment по актерам, режиссерам и жанрам

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from nltk.classify import NaiveBayesClassifier

In [2]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [19]:
df_train = df.loc[df['Genre'] != 'unknown']
df_train.Genre.unique() #2264
# len(df_train) #28803

array(['western', 'comedy', 'short', ...,
       'adventure, romance, fantasy film', 'ero',
       'horror romantic comedy'], dtype=object)

In [4]:
# Load and prepare the dataset
import nltk
from nltk.corpus import movie_reviews
import random

documents = [(list(movie_reviews.words(fileid)), category) for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

In [13]:
documents[0]

(['as',
  'any',
  'reasonable',
  'human',
  'being',
  'would',
  ',',
  'i',
  'must',
  'admit',
  'that',
  'occasionally',
  'i',
  'am',
  'befuddled',
  'by',
  'certain',
  'things',
  '.',
  'i',
  'am',
  'befuddled',
  'by',
  'fight',
  'club',
  '.',
  'now',
  'sometimes',
  'our',
  'favorite',
  'films',
  'are',
  'panned',
  'by',
  'critics',
  'and',
  'do',
  'not',
  'achieve',
  'critical',
  'success',
  'until',
  'much',
  'later',
  ',',
  'sometimes',
  'as',
  'much',
  'as',
  'ten',
  'or',
  'twenty',
  'years',
  'later',
  '.',
  'perhaps',
  'it',
  'is',
  'because',
  'these',
  'films',
  'are',
  'way',
  'ahead',
  'of',
  'their',
  'time',
  '.',
  'perhaps',
  'it',
  'is',
  'because',
  'they',
  'are',
  'so',
  'new',
  'and',
  'complex',
  'that',
  'only',
  'after',
  'years',
  'of',
  'allowing',
  'them',
  'to',
  'sink',
  'in',
  'do',
  'we',
  'truly',
  'appreciate',
  'and',
  'understand',
  'them',
  '.',
  'fight',
  'clu

In [12]:
documents[1]

(['plot',
  ':',
  'something',
  'about',
  'a',
  'bunch',
  'of',
  'kids',
  'going',
  'into',
  'a',
  'haunted',
  'house',
  'and',
  'playing',
  'out',
  'parodies',
  'of',
  'other',
  'horror',
  'and',
  'non',
  '-',
  'horror',
  'movies',
  '.',
  'oh',
  'yeah',
  ',',
  'there',
  "'",
  's',
  'also',
  'something',
  'about',
  'a',
  'ghost',
  'possessing',
  'the',
  'mansion',
  'and',
  'shit',
  ',',
  'but',
  'trust',
  'me',
  ',',
  'you',
  'won',
  "'",
  't',
  'care',
  'much',
  'about',
  'the',
  '"',
  'plot',
  '"',
  'in',
  'this',
  'film',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'parody',
  'movies',
  'either',
  'work',
  'or',
  'they',
  'don',
  "'",
  't',
  'and',
  'this',
  'one',
  'just',
  'doesn',
  "'",
  't',
  'work',
  '!',
  'the',
  'laughs',
  'aren',
  "'",
  't',
  'as',
  'pronounced',
  'this',
  'time',
  'around',
  ',',
  'the',
  'energy',
  'level',
  ',',
  'the',
  'gags',
  'and',
  'all',
  'of',
  'the',


In [5]:
# Define the feature extractor

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [6]:
# Train Naive Bayes classifier
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [7]:
# Test the classifier
print(nltk.classify.accuracy(classifier, test_set))

0.8


In [8]:
# Show the most important features as interpreted by Naive Bayes
classifier.show_most_informative_features(5)

Most Informative Features
   contains(outstanding) = True              pos : neg    =     10.2 : 1.0
         contains(mulan) = True              pos : neg    =      9.1 : 1.0
        contains(seagal) = True              neg : pos    =      8.1 : 1.0
         contains(damon) = True              pos : neg    =      7.7 : 1.0
   contains(wonderfully) = True              pos : neg    =      7.2 : 1.0
